### __LIBRARIES__

In [ ]:
import numpy as np
import os
import pandas as pd
import sys

path = "/Users/migue/Documents/F1 Data Center/F1-Data-Center/Programs/Python/"

sys.path.append(path)
import multidata



__DATA LOADING__

In [ ]:
path = "/Users/migue/Documents/F1 Data Center/Datos/"
os.chdir(path)

df = pd.read_csv('data.csv', sep = ';', encoding = "ISO-8859-1")

display(df)

colNames: list = df.columns.values.tolist() #Column names
print(colNames)

__YEARS SET__

In [ ]:
year = 2019

#Data of the corresponding day #TODO IN CASE OF ALL YEARS
if year != 'all':
    index_year = df['Year'] == year
    df_Year = df[index_year]
    display(df_Year)
else:
    df_Year = df
    display(df_Year)


__POINTS BASED ON FREE PRACTICE AND QUALIFYING (BY YEAR)__ 

In [ ]:
drivers = list(df_Year["Driver"].unique()) #Drivers participating in the corresponding time period
n_gps = list(df_Year["GP"].unique()) #GPs during the corresponding time period

colors = multidata.f1_teams(drivers, year, 'Default', 'Default').color  #Colors of each team deppending 

#Times in each GP and each driver and the corresponding points obtained during that period
fp1_times = np.zeros((len(n_gps),len(drivers)))
fp1_points = np.zeros((len(n_gps),len(drivers)))

fp2_times = np.zeros((len(n_gps),len(drivers)))
fp2_points = np.zeros((len(n_gps),len(drivers)))

fp3_times = np.zeros((len(n_gps),len(drivers)))
fp3_points = np.zeros((len(n_gps),len(drivers)))

quali_times = np.zeros((len(n_gps),len(drivers)))
quali_points = np.zeros((len(n_gps),len(drivers)))

race_pos = np.zeros((len(n_gps),len(drivers)))
race_points = np.zeros((len(n_gps),len(drivers)))

yearData = df_Year.to_numpy()
driver_name = colNames.index("Driver")
gp_index = colNames.index("GP")

fp1 = colNames.index("fp1_fastest_lap")
fp2 = colNames.index("fp2_fastest_lap")
fp3 = colNames.index("fp3_fastest_lap")
quali = colNames.index("quali_fastest_lap")
race = colNames.index("race_Result")


for row in yearData:
    gpName = row[gp_index] #GP on this row
    driv = row[driver_name] #Driver on this row

    i = n_gps.index(gpName)
    j = drivers.index(driv)

    fp1_times[i,j] = row[fp1]
    fp2_times[i,j] = row[fp2]
    fp3_times[i,j] = row[fp3]
    quali_times[i,j] = row[quali]
    if row[race] == 'DNF': race_pos[i,j] = 0
    else:  race_pos[i,j] = row[race]


Compute the order of the drivers in each GP

In [ ]:
points = [25,18,15,12,10,8,6,4,2,1,0,0,0,0,0,0,0,0,0,0]

sesion = [fp1_times.copy(),fp2_times.copy(),fp3_times.copy(),quali_times.copy(), race_pos.copy()] #Include every Sesion time (4, len(gPs), len(drivers))
ses_points = [fp1_points.copy(),fp2_points.copy(),fp3_points.copy(),quali_points.copy(), race_points.copy()] #Include every Sesion points (4, len(gPs), len(drivers))

for ses_index, s in enumerate(sesion):
    for circ_index, circuit in enumerate(s):
        p = -1
        if sum(circuit) > 10:
            while circuit[np.argmin(circuit)] < sys.maxsize:
                minim = np.argmin(circuit)
                if circuit[minim] == 0:
                    circuit[minim] = sys.maxsize
                else:
                    p += 1
                    minim =np.argmin(circuit)
                    circuit[minim] = sys.maxsize
                    ses_points[ses_index][circ_index,minim] = points[p]

print(ses_points[4][:,drivers.index('Valtteri BOTTAS')])

__Total Points__ 

In [ ]:
titles = ['FP1 TOTAL POINTS', 'FP2 TOTAL POINTS','FP3 TOTAL POINTS', 'QUALIFYING TOTAL POINTS', 'RACE TOTAL POINTS']
totalPoints = np.zeros((len(ses_points),len(drivers))) #Reduce the total to (4, len(driveres)) matrix which incluedes the total amount of points scored by each driver in each Sesion

for ses_index, s in enumerate(ses_points):
    for driv_index, driver in enumerate(drivers):
        totalPoints[ses_index,driv_index] = sum(s[circ_index,driv_index] for circ_index, circuit in enumerate(s))


__Year Progression__

In [ ]:
yearProgessionOK = np.zeros((len(ses_points),len(drivers),len(n_gps))) #Each sesion, and evolution of each driver points per race 

for ses_index, s in enumerate(ses_points):
    for gpInd, gp in enumerate(n_gps):
        for driv_index, driver in enumerate(drivers):
            totalUntilGP = sum(s[circ_index,driv_index] for circ_index in range(gpInd+1))
            yearProgessionOK[ses_index,driv_index, gpInd] = totalUntilGP

__Data Organization__

In [ ]:
def orderLists(points: np.array, driverName: list, color: list):
    '''Necessary to plot, orders the list in function on the first introduced list. LISTS MUST HAVE THE SAME SIZE'''
    newPoints, newDriv, newColor = [], [], []
    while points[np.argmax(points)] >= 0:
        maxim =  np.argmax(points)

        newPoints.append(points[maxim])
        newDriv.append(driverName[maxim])
        newColor.append(color[maxim])
            
        points[maxim] = -1

    return newPoints, newDriv, newColor

__PLOTS__

In [ ]:
import matplotlib.pyplot as plt

def driv_min(drivers):
    driv_min = []
    for d in drivers:
        surname = d.split()[-1]
        if surname[0:3] not in driv_min:
            driv_min.append(surname[0:3].upper())
        else:
            cond = True
            while cond:
                x = 4
                if surname[0:x] not in driv_min:
                    cond = False
                    driv_min.append(surname[0:3].upper())
                else:
                    x += 1

    return driv_min

def addlabels(y, colors):
    for i, v in enumerate(sorted(y, reverse=True)):
        plt.text(v+0.2, i, str(int(v)), color='#090909', va="center")

drivers_dim = driv_min(drivers.copy())
for s_index,sesion in enumerate(totalPoints):
    ses,drivers_dimPlot,colorTeam = orderLists(sesion.copy()[:22],drivers_dim.copy()[:22],colors.copy()[:22])
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1]) 

    ax.barh(drivers_dimPlot,ses,color = colorTeam)
    addlabels(ses, colorTeam)
    
    ax.set_facecolor('#DFF0FF')

    plt.title(titles[s_index]) 
    plt.xlabel("Points")
    plt.ylabel("Drivers")

    plt.gca().invert_yaxis()
    plt.xticks(range(0, int(ses[0]), 50))
    plt.show()

__Year Progression Plot__

In [ ]:
def maxValuePlot(sesion: list):
    tot = [d[-1] for d in sesion]
    return max(tot)

def orderByTeam(yearProgessionOK: np.array, colorsPlot: list, driversPlot: list):
    yearProgession = []
    colors = []
    drivers = []

    yearProgessionTeams = []
    colorsTeams = []
    teams = []
    for s_index,sesion in enumerate(yearProgessionOK):
        ses = []
        col = []
        drivList = []

        sesTeams = []
        colTeams = []
        teamsList = []        

       
        teamsName = multidata.f1_teams(driversPlot[s_index], year, 'Default', 'Default').equipos
        teamsCol = list(set(colorsPlot[s_index]))
        
        points = [driver[-1] for driv_index, driver in enumerate(sesion)]
        teamIndexes = []
        teamPoints = []

        for t in teamsCol:
            teamIndexes.append([i for i in range(len(colorsPlot[s_index])) if colorsPlot[s_index][i] == t])
            teamPoints.append(sum(points[p] for p in range(len(points)) if colorsPlot[s_index][p] == t))
        
        orderedIndex = []
        originalLen = len(teamPoints)
        while len(orderedIndex) != originalLen:
            index = teamPoints.index(max(teamPoints))
            orderedIndex.append(teamIndexes[index])
            teamPoints[index] = -1

        for t, teamIndexArray in enumerate(orderedIndex):
            total = np.zeros(sesion[0].shape) #MUST BE THE TOTAL NUMBER OF RACES DURING A YEAR
            for d in teamIndexArray:
                #DRIVERS
                ses.append(sesion[d])
                col.append(colorsPlot[s_index][d])
                drivList.append(driversPlot[s_index][d])

                #TEAMS TOTAL
                total += sesion[d]

            #TEAMS
            sesTeams.append(total)
            colTeams.append(colorsPlot[s_index][d])
            teamsList.append(teamsName[d])

        yearProgession.append(ses)
        colors.append(col)
        drivers.append(drivList)

        yearProgessionTeams.append(sesTeams)
        colorsTeams.append(colTeams)
        teams.append(teamsList)

    return yearProgession, colors, drivers, yearProgessionTeams, colorsTeams, teams

#REMOVE 0s drivers for a better visibility
colorsPlot = []
driversPlot = []

yearProgession = []
for s_index,sesion in enumerate(yearProgessionOK):
    ses = []
    col = []
    drivList = []
    for driv_index, driver in enumerate(sesion):
        if driver[-1] != 0: # 0 in the last GP
            ses.append(driver)
            col.append(colors[driv_index])
            drivList.append(drivers[driv_index])

    yearProgession.append(ses)
    colorsPlot.append(col)
    driversPlot.append(drivList)

yearProgession, colorsPlot, driversPlot, yearProgessionTeams, colorsTeams, teams = orderByTeam(yearProgession.copy(), colorsPlot.copy(), driversPlot.copy())


x_axes = [i for i in range(len(n_gps))]
for s_index,sesion in enumerate(yearProgession):
    teams_av = []
    teams_driv = multidata.f1_teams(driversPlot[s_index], year, 'Default', 'Default').equipos
    drivers_dim = driv_min(driversPlot[s_index].copy())
    for driv_index, driver in enumerate(sesion):
        if teams_driv[driv_index] not in teams_av:
            teams_av.append(teams_driv[driv_index])
            plt.plot(x_axes, driver, linewidth=2 , color = colorsPlot[s_index][driv_index], label = drivers_dim[driv_index]) 
        else:
            plt.plot(x_axes, driver, linewidth=2 , linestyle = 'dashdot', color = colorsPlot[s_index][driv_index], label =drivers_dim[driv_index])


    
    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_facecolor('#DFF0FF')
    plt.title(titles[s_index]) 
    plt.xlabel("GP")
    plt.ylabel("Total Points")
    plt.xticks(range(0, int(len(n_gps)), 1))
    plt.yticks(range(0, int(maxValuePlot(sesion) + 25), 25))
    plt.show()

__Teams Progression Plot__

In [ ]:
x_axes = [i for i in range(len(n_gps))]
for s_index,sesion in enumerate(yearProgessionTeams):
    # drivers_dim = driv_min(driversPlot[s_index].copy())
    for team_index, team in enumerate(sesion):
        plt.plot(x_axes, team, linewidth=2 , color = colorsTeams[s_index][team_index], label = teams[s_index][team_index]) 
    
    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_facecolor('#DFF0FF')
    plt.title(titles[s_index]) 
    plt.xlabel("GP")
    plt.ylabel("Total Points")
    plt.xticks(range(0, int(len(n_gps)), 1))
    plt.yticks(range(0, int(maxValuePlot(sesion) + 50), 50))
    plt.show()